In [13]:
import collections

# --- Trie Node and Base Class ---
class TrieNode:
    def __init__(self):
        self.children = collections.defaultdict(TrieNode)
        self.is_end_of_word = False
        self.count = 0

class Trie:
    def __init__(self):
        self.root = TrieNode()

    def insert(self, word: str):
        node = self.root
        for char in word:
            node = node.children[char]
            node.count += 1
        node.is_end_of_word = True

    def find_stem_suffix(self, word: str) -> (str, str):
        node = self.root
        best_split_point = 0
        max_score = -1

        for i, char in enumerate(word):
            if char not in node.children:
                return word, ""
            node = node.children[char]
            
            # A node is a potential split point if it branches OR it's a valid word itself.
            is_potential_split = (len(node.children) > 1 or node.is_end_of_word)
            
            # The score is the node's frequency. Higher frequency means a more common stem.
            score = node.count

            # We look for the best score at a potential split point.
            if is_potential_split and score > max_score and i < len(word) - 1:
                max_score = score
                best_split_point = i + 1
        
        if best_split_point == 0:
            return word, ""

        stem = word[:best_split_point]
        suffix = word[best_split_point:]
        return stem, suffix

# --- Prefix Trie Implementation ---
class PrefixTrie(Trie):
    def analyze_word(self, word: str):
        return self.find_stem_suffix(word)

# --- Suffix Trie Implementation ---
class SuffixTrie(Trie):
    def insert(self, word: str):
        super().insert(word[::-1])

    def analyze_word(self, word: str):
        reversed_word = word[::-1]
        rev_head, rev_tail = self.find_stem_suffix(reversed_word)
        
        # The reversed "head" of the reversed word is the suffix
        suffix = rev_head[::-1]
        # The reversed "tail" of the reversed word is the stem
        stem = rev_tail[::-1]
        
        return stem, suffix

# --- Main Execution ---
def run_full_analysis(filename="brown_nouns.txt"):
    """
    Reads all nouns from the file, analyzes them with both tries,
    and prints the complete results.
    """
    try:
        with open(filename, 'r') as f:
            words_from_file = [line.strip().lower() for line in f if line.strip()]

            # --- THE FIX IS HERE ---
            # Filter the list to include only alphabetic words
            words = sorted(list(set([
                word for word in words_from_file if word.isalpha()
            ])))
            # --- END OF FIX ---

    except FileNotFoundError:
        print(f"Error: '{filename}' not found. Please place it in the same directory.")
        return

    # (The rest of the function remains the same)
    ...

    # Initialize and populate tries
    prefix_trie = PrefixTrie()
    suffix_trie = SuffixTrie()
    for word in words:
        prefix_trie.insert(word)
        suffix_trie.insert(word)

    print("--- Full Analysis Results from brown_nouns.txt ---")
    print("Word".ljust(20), "PrefixTrie".ljust(25), "SuffixTrie".ljust(25))
    print("-" * 70)

    # Iterate over the complete list of words from the file
    for word in words:
        p_stem, p_suffix = prefix_trie.analyze_word(word)
        s_stem, s_suffix = suffix_trie.analyze_word(word)
        
        p_out = f"{p_stem}+{p_suffix}" if p_suffix else p_stem
        s_out = f"{s_stem}+{s_suffix}" if s_suffix else s_stem
        
        print(word.ljust(20), p_out.ljust(25), s_out.ljust(25))


if __name__ == "__main__":
    run_full_analysis()

--- Full Analysis Results from brown_nouns.txt ---
Word                 PrefixTrie                SuffixTrie               
----------------------------------------------------------------------
aaa                  a+aa                      aa+a                     
abandon              a+bandon                  abando+n                 
abandonment          a+bandonment              abandonmen+t             
abaringe             a+baringe                 abaring+e                
abasement            a+basement                abasemen+t               
abberations          a+bberations              abberation+s             
abbey                a+bbey                    abbe+y                   
abbot                a+bbot                    abbo+t                   
abbreviation         a+bbreviation             abbreviatio+n            
abbreviations        a+bbreviations            abbreviation+s           
abdomen              a+bdomen                  abdome+n                 
ab

wiht probability measure

In [15]:
import collections
import math

# --- Trie Node and Base Class (No change) ---
class TrieNode:
    def __init__(self):
        self.children = collections.defaultdict(TrieNode)
        self.is_end_of_word = False
        self.count = 0

class Trie:
    def __init__(self):
        self.root = TrieNode()

    def insert(self, word: str):
        node = self.root
        for char in word:
            node = node.children[char]
            node.count += 1
        node.is_end_of_word = True

    def _calculate_entropy(self, node: TrieNode) -> float:
        if not node.children or node.count == 0:
            return 0.0
        entropy = 0.0
        for child in node.children.values():
            probability = child.count / node.count
            if probability > 0:
                entropy -= probability * math.log2(probability)
        return entropy

    def find_stem_suffix(self, word: str) -> (str, str, float): # MODIFIED return type
        node = self.root
        best_split_point = 0
        max_score = -1.0

        for i, char in enumerate(word):
            if char not in node.children:
                return word, "", 0.0 # Return 0 score if not found
            node = node.children[char]
            
            is_potential_split = (len(node.children) > 1 or node.is_end_of_word)
            
            if is_potential_split and i < len(word) - 1:
                frequency = node.count
                entropy = self._calculate_entropy(node)
                score = frequency * (entropy + 1)
                if score > max_score:
                    max_score = score
                    best_split_point = i + 1
        
        if best_split_point == 0:
            return word, "", 0.0 # Return 0 score if no split

        stem = word[:best_split_point]
        suffix = word[best_split_point:]
        return stem, suffix, max_score # MODIFIED: Return the score

# --- PrefixTrie and SuffixTrie (Modified to handle score) ---
class PrefixTrie(Trie):
    def analyze_word(self, word: str):
        # Now returns stem, suffix, and the score
        return self.find_stem_suffix(word)

class SuffixTrie(Trie):
    def insert(self, word: str):
        super().insert(word[::-1])

    def analyze_word(self, word: str):
        reversed_word = word[::-1]
        rev_head, rev_tail, score = self.find_stem_suffix(reversed_word) # Capture score
        suffix = rev_head[::-1]
        stem = rev_tail[::-1]
        return stem, suffix, score # Pass score through

# --- Main Execution (Modified to print score) ---
def run_analysis_on_test_set():
    test_words = [
        "go", "goes", "do", "does", "kite", "kites", "bus", "buses",
        "city", "cities", "try", "tries", "car", "cars", "boy", "boys",
        "apple", "apples", "play", "playing", "player", "players"
    ]
    
    prefix_trie = PrefixTrie()
    suffix_trie = SuffixTrie()
    for word in test_words:
        prefix_trie.insert(word)
        suffix_trie.insert(word)

    print("--- Analysis with Score Displayed ---")
    print("Word".ljust(15), "PrefixTrie (Score)".ljust(25), "SuffixTrie (Score)".ljust(25))
    print("-" * 65)

    words_to_analyze = ["goes", "kites", "cities", "cars", "apples", "player", "players", "playing"]
    for word in sorted(list(set(words_to_analyze))):
        p_stem, p_suffix, p_score = prefix_trie.analyze_word(word)
        s_stem, s_suffix, s_score = suffix_trie.analyze_word(word)
        
        p_out = f"{p_stem}+{p_suffix}" if p_suffix else p_stem
        s_out = f"{s_stem}+{s_suffix}" if s_suffix else s_stem
        
        print(f"{word:<15} {p_out:<15} ({p_score:7.2f}) {s_out:<15} ({s_score:7.2f})")

if __name__ == "__main__":
    run_analysis_on_test_set()

--- Analysis with Score Displayed ---
Word            PrefixTrie (Score)        SuffixTrie (Score)       
-----------------------------------------------------------------
apples          apple+s         (   3.00) apple+s         (  27.40)
cars            c+ars           (   8.00) car+s           (  27.40)
cities          c+ities         (   8.00) citie+s         (  27.40)
goes            go+es           (   3.00) goe+s           (  27.40)
kites           kite+s          (   3.00) kite+s          (  27.40)
player          play+er         (   8.00) playe+r         (   4.00)
players         play+ers        (   8.00) player+s        (  27.40)
playing         play+ing        (   8.00) +playing        (   0.00)
